In [90]:
import numpy as np
import keras
from keras import backend as K
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
import itertools

In [91]:
train_path=r'C:\Users\kushagra\KAGGLE\Untitled Folder\Untitled Folder\cat-and-dog\training_set'
test_path=r'C:\Users\kushagra\KAGGLE\Untitled Folder\Untitled Folder\cat-and-dog\test_set'

In [92]:
train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['dogs','cats'],batch_size=10)
test_batches=ImageDataGenerator().flow_from_directory(test_path,target_size=(224,224),classes=['dogs','cats'],batch_size=10)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


BUID AND TRAIN CNN

In [93]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    Flatten(),
    Dense(2,activation='softmax')

])

In [94]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [95]:
model.fit_generator(train_batches,steps_per_epoch=1,epochs=5,verbose=2)

Epoch 1/5
 - 1s - loss: 8.0590 - acc: 0.5000
Epoch 2/5
 - 0s - loss: 9.6709 - acc: 0.4000
Epoch 3/5
 - 0s - loss: 4.8354 - acc: 0.7000
Epoch 4/5
 - 0s - loss: 8.0590 - acc: 0.5000
Epoch 5/5
 - 0s - loss: 11.2827 - acc: 0.3000


In [96]:
predictions=model.predict_generator(test_batches,steps=1,verbose=0)

In [97]:
predictions

array([[1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 2.8919752e-29],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 2.7037461e-19],
       [1.0000000e+00, 2.3728722e-38],
       [1.0000000e+00, 0.0000000e+00]], dtype=float32)

BUILT FINE TUNED VGG16 MODEL

In [98]:
vgg16_model = keras.applications.vgg16.VGG16()

In [99]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [100]:
type(vgg16_model)

keras.engine.training.Model

In [101]:
model=Sequential()


In [102]:
for layer in vgg16_model.layers:
    model.add(layer)

In [103]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)     

In [104]:
model.layers.pop()

In [105]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)     

In [106]:
for layer in model.layers:
    layer.trainable=False

In [107]:
model.add(Dense(2,activation='softmax'))

In [108]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)     

TRAIN THE FINE TUNED VGG 16 MODEL

In [109]:
model.compile(Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [110]:
model.fit_generator(train_batches,steps_per_epoch=1,epochs=5,verbose=2)

Epoch 1/5
 - 1s - loss: 0.6968 - acc: 0.5000
Epoch 2/5
 - 0s - loss: 0.6821 - acc: 0.7000
Epoch 3/5
 - 0s - loss: 0.6924 - acc: 0.5000
Epoch 4/5
 - 0s - loss: 0.6807 - acc: 0.7000
Epoch 5/5
 - 0s - loss: 0.6743 - acc: 0.9000


In [111]:
predictions=model.predict_generator(test_batches,steps=1,verbose=0)


In [112]:
predictions

array([[0.50011295, 0.49988705],
       [0.5075563 , 0.49244362],
       [0.47975615, 0.5202438 ],
       [0.500458  , 0.499542  ],
       [0.50652933, 0.49347064],
       [0.5003529 , 0.49964705],
       [0.50471735, 0.4952827 ],
       [0.506838  , 0.49316198],
       [0.51220906, 0.48779088],
       [0.5038707 , 0.49612924]], dtype=float32)